In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from AgentNet import AgentNetManager
from AgentNet.Agent import load_ignore_patterns,generate_tree_structure,generate_ssh_grounding
from AgentNet.config import python_ssh_client_mariana

In [6]:
python_ssh_client_mariana.connect()

In [7]:
manager = AgentNetManager(project="DFT_AgentTest3")

def update_grounding():
    # Load the ignore patterns
    ignore_patterns = load_ignore_patterns(".ignore")
    # Generate the tree structure for the current directory
    local_grounding = generate_tree_structure(".", ignore_patterns)
    virtual_grounding = generate_ssh_grounding(python_ssh_client_mariana)
    grounding = "###### Local Working Directory #####\n"+ local_grounding +"\n" + "###### Files Inside HPC #####\n" + virtual_grounding
    return grounding


manager.add_grounding(grounding_context="Here are the files in your working directory and HPC working directory",grounding=update_grounding)


In [8]:
model = "gpt-4o"
# Configuration Recommender
configuration_recommender = manager.create_procedure_node("Configuration_Recommender", description = "Recommend DFT configurations such as basis set and funtional for molecules, before you terminate, you must provide Thought: the basis set is <recommended basis set> and the functional is <recommended functional>",model=model)
basis_set_recommender = manager.create_procedure_node("Basis_Set_Recommender", description = "Recommend basis set for DFT calculations. Search for the basis set for this molecule",model=model)
functional_recommender = manager.create_procedure_node("Functional_Recommender", description = "Recommend functional for DFT calculations. Search for the functional for this molecule",model=model)
tool_websearch = manager.create_tool_node("perplexity_ai_search", description="Provide detailed answers that answers questions based on accurate internet and scientific resources")

# Connect configuration -> basis set, functional. Connect basis set, functional -> tool_websearch
manager.create_edge(from_=configuration_recommender, to=basis_set_recommender)
manager.create_edge(from_=configuration_recommender, to=functional_recommender)
manager.create_edge(from_=basis_set_recommender, to=tool_websearch)
manager.create_edge(from_=functional_recommender, to=tool_websearch)



AgentNetProcedureEdge(from_=AgentNetProcedureNode(label='Functional_Recommender', description='Recommend functional for DFT calculations. Search for the functional for this molecule', entity=<AgentNet.Agent.langgraph_react_agent_objective_output.LangGraphAgentReactExperimental object at 0x000001A3B2399B20>), to=AgentNetToolNode(label='perplexity_ai_search', description='Provide detailed answers that answers questions based on accurate internet and scientific resources', entity=StructuredTool(name='perplexity_ai_search', description='Search the web for information\n\nArgs:\n    query (str): The query to search for', args_schema=<class 'langchain_core.utils.pydantic.perplexity_ai_search'>, func=<function perplexity_ai_search at 0x000001A3B24EBC40>)))

In [9]:
input_file_generator = manager.create_procedure_node("Input_File_Generator", description = """Generate ORCA input file. Step 1: Propose the ORCA tool. Step 2: Use Python Terminal to create a copy of the proposed file. Include the proposed file in the input context when propose actions to use python to create the input file""",model=model)

tool_orca_proposal = manager.create_tool_node("orca_proposal", description="Propose content of the ORCA input file. Input: configuration, xyz filename, objective for calculation.")
tool_python = manager.create_tool_node("python_repl", description="Python Terminal that run python code") 
#tool_read_file_content = manager.create_tool_node("read_file_content", description="Read the content of the input file")
# Gnerator -> Proposer, Python
manager.create_edge(from_=input_file_generator, to=tool_python)
manager.create_edge(from_=input_file_generator, to=tool_orca_proposal)

AgentNetProcedureEdge(from_=AgentNetProcedureNode(label='Input_File_Generator', description='Generate ORCA input file. Step 1: Propose the ORCA tool. Step 2: Use Python Terminal to create a copy of the proposed file. Include the proposed file in the input context when propose actions to use python to create the input file', entity=<AgentNet.Agent.langgraph_react_agent_objective_output.LangGraphAgentReactExperimental object at 0x000001A3B1AFE480>), to=AgentNetToolNode(label='orca_proposal', description='Propose content of the ORCA input file. Input: configuration, xyz filename, objective for calculation.', entity=StructuredTool(name='orca_proposal', description='Propose content of the ORCA input file. When used, provide recommended configuration, xyz file and objective for calculation', args_schema=<class 'langchain_core.utils.pydantic.orca_proposal'>, func=<function orca_proposal at 0x000001A3B3B5A200>)))

In [ ]:
manager.compile()

Node Configuration_Recommender connected
Node Basis_Set_Recommender connected
Node Functional_Recommender connected
Node Input_File_Generator connected
Compilation complete


In [ ]:
manager.stream(input_node_label="Input_File_Generator",query="Create an input file with the following configuration: basis set is 6-31G and functional is B3LYP, the xyz file called xtb_optimized.xyz, objective is use DFT to optimize the gemoetry, use PythonREPL to run the python")

In [10]:
# File Manger
file_manager = manager.create_procedure_node("File_Manager", description = "Manage files in the working and HPC directory. File_Manager will be asked to transfer files between local and remote HPC directory. File_Manager can read file or search keywords in files. Remember you can only read local file. To read HPC file, you always need to transfer file from HPC to local and then read it",model=model)
tool_send_file_to_hpc = manager.create_tool_node("send_file_to_hpc", description="Send file to HPC working directory")
tool_send_file_to_local = manager.create_tool_node("send_file_to_local", description="Send file from HPC to local directory")
tool_hpc_shell = manager.create_tool_node("hpc_shell", description="Execute shell command on HPC, can be used to run orca command as well as os operations for HPC files")
tool_search_by_keyword = manager.create_tool_node("search_by_keywords", description="Search for keyword in the file content,NOTE it only deal with local files")
tool_read_file_content = manager.create_tool_node("read_file_content", description="Read the content of the file, NOTE it only deal with local files")
# File Manager -> Send File to HPC, Send File to Local, HPC Shell
manager.create_edge(from_=file_manager, to=tool_send_file_to_hpc)
manager.create_edge(from_=file_manager, to=tool_send_file_to_local)
manager.create_edge(from_=file_manager, to=tool_hpc_shell)
manager.create_edge(from_=file_manager, to=tool_search_by_keyword)
manager.create_edge(from_=file_manager, to=tool_read_file_content)


AgentNetProcedureEdge(from_=AgentNetProcedureNode(label='File_Manager', description='Manage files in the working and HPC directory. File_Manager will be asked to transfer files between local and remote HPC directory. File_Manager can read file or search keywords in files. Remember you can only read local file. To read HPC file, you always need to transfer file from HPC to local and then read it', entity=<AgentNet.Agent.langgraph_react_agent_objective_output.LangGraphAgentReactExperimental object at 0x000001A3B238FC80>), to=AgentNetToolNode(label='read_file_content', description='Read the content of the file, NOTE it only deal with local files', entity=StructuredTool(name='read_file_content', description='Reads and returns the content of a specified file as a string.\n\nArgs:\n    file_path (str): Path to the file to be read.\n\nReturns:\n    str: The content of the file.', args_schema=<class 'langchain_core.utils.pydantic.read_file_content'>, func=<function read_file_content at 0x00000

In [10]:
manager.stream(input_node_label="File_Manager",query="get the input file from remote please")

Agent:  File_Manager
Thought: I need to transfer the input file from the remote HPC directory to the local working directory., 
 Action: objective: Transfer a file from remote HPC to local directory, input_context: Transfer the file named 'input.inp' from the HPC directory to the local working directory., output: The file 'input.inp' will be available in the local working directory.,
 Terminate: False
File '/u/yunhengzou/el-agente/input.inp' successfully transferred to 'input.inp' on the local machine.
Observation: The file 'input.inp' has been successfully transferred from the HPC to your local working directory.
Thought: The input file has been successfully transferred to the local directory., 
 Action: message: objective achieved,
 Terminate: True


[AIMessage(content="Thought: I need to transfer the input file from the remote HPC directory to the local working directory., \n Action: objective: Transfer a file from remote HPC to local directory, input_context: Transfer the file named 'input.inp' from the HPC directory to the local working directory., output: The file 'input.inp' will be available in the local working directory.,\n Terminate: False", additional_kwargs={}, response_metadata={}, id='c6c1a029-6df1-4e8b-a4be-86da446f9d0d'),
 HumanMessage(content="Observation: The file 'input.inp' has been successfully transferred from the HPC to your local working directory.", additional_kwargs={}, response_metadata={}, id='1f0ecde1-9243-4703-8a8e-c9653ee05942'),
 AIMessage(content='Thought: The input file has been successfully transferred to the local directory., \n Action: message: objective achieved,\n Terminate: True', additional_kwargs={}, response_metadata={}, id='3ac42cc2-7dda-4857-a79f-0c2bdb294666')]

In [11]:
# ORCA Runner
orca_runner = manager.create_procedure_node("ORCA_Runner", description = "Run ORCA calculations on HPC. You will be given input file and you should generate shell command. Make sure it have output file. If it fails, you must immediately terminate. You should not debug yourself",model=model)
tool_hpc_shell = manager.create_tool_node("hpc_shell", description="Execute shell command on HPC, can be used to run orca command as well as os operations for HPC files")
# ORCA Runner -> HPC Shell
manager.create_edge(from_=orca_runner, to=tool_hpc_shell)

Tool node label='hpc_shell' description='Execute shell command on HPC, can be used to run orca command as well as os operations for HPC files' entity=StructuredTool(name='hpc_shell', description='Execute shell command on the HPC.\nThe environment has ORCA installed. \nThe shell is already at working directory', args_schema=<class 'langchain_core.utils.pydantic.hpc_shell'>, func=<function hpc_shell at 0x000001A3B3B59080>) already exists


AgentNetProcedureEdge(from_=AgentNetProcedureNode(label='ORCA_Runner', description='Run ORCA calculations on HPC. You will be given input file and you should generate shell command. Make sure it have output file. If it fails, you must immediately terminate. You should not debug yourself', entity=<AgentNet.Agent.langgraph_react_agent_objective_output.LangGraphAgentReactExperimental object at 0x000001A3B23B5520>), to=AgentNetToolNode(label='hpc_shell', description='Execute shell command on HPC, can be used to run orca command as well as os operations for HPC files', entity=StructuredTool(name='hpc_shell', description='Execute shell command on the HPC.\nThe environment has ORCA installed. \nThe shell is already at working directory', args_schema=<class 'langchain_core.utils.pydantic.hpc_shell'>, func=<function hpc_shell at 0x000001A3B3B59080>)))

In [ ]:
manager.stream(input_node_label="ORCA_Runner",query="Run ORCA calculations on HPC. You only need to do orca xxx.input > xxx.output, if error encountered. TERMINATE Immediately")

In [12]:
# DFT Agent
DFT_Agent = manager.create_procedure_node("DFT_Agent", 
                                          description = """You orchestrate a team to perform DFT calculation. Note that you must transfer both xyz file and input file to HPC to run. To read any file, you must ask file manager to get the file from HPC to local computer in order to read it. Always read Output file to confirm execution as output file will store errors and results.""",model=model)
manager.create_edge(from_=DFT_Agent, to=configuration_recommender)
manager.create_edge(from_=DFT_Agent, to=input_file_generator)
manager.create_edge(from_=DFT_Agent, to=file_manager)
manager.create_edge(from_=DFT_Agent, to=orca_runner)


AgentNetProcedureEdge(from_=AgentNetProcedureNode(label='DFT_Agent', description='You orchestrate a team to perform DFT calculation. Note that you must transfer both xyz file and input file to HPC to run. To read any file, you must ask file manager to get the file from HPC to local computer in order to read it. Always read Output file to confirm execution as output file will store errors and results.', entity=<AgentNet.Agent.langgraph_react_agent_objective_output.LangGraphAgentReactExperimental object at 0x000001A3B2337FB0>), to=AgentNetProcedureNode(label='ORCA_Runner', description='Run ORCA calculations on HPC. You will be given input file and you should generate shell command. Make sure it have output file. If it fails, you must immediately terminate. You should not debug yourself', entity=<AgentNet.Agent.langgraph_react_agent_objective_output.LangGraphAgentReactExperimental object at 0x000001A3B23B5520>))

In [13]:
manager.compile()

Node Configuration_Recommender connected
Node Basis_Set_Recommender connected
Node Functional_Recommender connected
Node Input_File_Generator connected
Node File_Manager connected
Node ORCA_Runner connected
Node DFT_Agent connected
Compilation complete


In [14]:
manager.stream(input_node_label="DFT_Agent",query="Optimize the xtb_optimized.xyz file using DFT calculations, generate the dft_optimized xyz file, the molecule inside is CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1.  ")

Agent:  DFT_Agent
Thought: I need to recommend DFT configurations for the molecule in xtb_optimized.xyz file., 
 Action: objective: Recommend DFT configurations for the molecule., input_context: The molecule is CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1., output: Recommended DFT configurations including basis set and functional.,
 Terminate: False
[Configuration_Recommender] ...
Thought: I need to recommend a basis set for the given molecule., 
 Action: objective: Recommend basis set for DFT calculations, input_context: Molecule: CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1, output: Recommended basis set for the molecule,
 Terminate: False
[Basis_Set_Recommender] ...
Thought: I need to find a suitable basis set for DFT calculations for the given molecule., 
 Action: objective: Find a suitable basis set for DFT calculations for the given molecule, input_context: Molecule: CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1, output: A recommended basis set for DFT calculations,
 Terminate: False
For Density Functional Theory

22:06:08.261 | WARNING | langchain_experimental.utilities.python - Python REPL can execute arbitrary code. Use with caution.

Deleted 6 documents from 'checkpoints' collection for thread_id 'ReACT_executor9eea54f1-ba41-4c59-b288-7a12c4f9c283'.
Deleted 0 documents from 'checkpoint_writes' collection for thread_id 'ReACT_executor9eea54f1-ba41-4c59-b288-7a12c4f9c283'.
Observation: A copy of the ORCA input file has been successfully created in the working directory with the specified content:

```
! B3PW91 def2-TZVP Opt

* xyzfile 0 1 rdkit_generated.xyz
```
Thought: The ORCA input file has been successfully created with the specified content., 
 Action: message: Objective achieved,
 Terminate: True
Deleted 11 documents from 'checkpoints' collection for thread_id 'Input_File_Generator4d2efec5-f71f-43ee-a9a2-01dc932b9a98'.
Deleted 6 documents from 'checkpoint_writes' collection for thread_id 'Input_File_Generator4d2efec5-f71f-43ee-a9a2-01dc932b9a98'.
[Input_File_Generator] Finished!
Deleted 6 documents from 'checkpoints' collection for thread_id 'ReACT_executor4fe414a5-f092-46bd-bba1-31cabaa10767'.
Deleted 0 docum

[AIMessage(content='Thought: I need to recommend DFT configurations for the molecule in xtb_optimized.xyz file., \n Action: objective: Recommend DFT configurations for the molecule., input_context: The molecule is CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1., output: Recommended DFT configurations including basis set and functional.,\n Terminate: False', additional_kwargs={}, response_metadata={}, id='7d64563a-df5b-4199-8583-4671d9b4c993'),
 HumanMessage(content='Observation: For the molecule CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1, the recommended DFT configurations are:\n\n- **Basis Set**: def2-TZVP\n- **Functionals**:\n  1. **B3PW91**: Known for good performance in predicting electron affinities and redox potentials.\n  2. **PBE0**: Reliable for a wide range of properties, including geometries and energies.\n  3. **M06**: Performs well in main-group thermochemistry, thermochemical kinetics, and noncovalent interactions.\n  4. **TPSSh**: Recommended for computing redox potentials, with performance s

In [15]:
manager.stream(input_node_label="DFT_Agent",query="You didn't transfer the xyz file, transfer the xyz file and do again")

Agent:  DFT_Agent
Thought: I need to transfer the xyz file to the HPC for the DFT calculations., 
 Action: objective: Transfer xyz file to HPC for DFT calculations., input_context: The xyz file is xtb_optimized.xyz and needs to be transferred to HPC., output: xyz file is transferred to HPC.,
 Terminate: False
[File_Manager] ...
Thought: I need to transfer the xtb_optimized.xyz file from the local directory to the HPC directory for DFT calculations., 
 Action: objective: Transfer a file from local to HPC directory, input_context: Transfer xtb_optimized.xyz from local to HPC, output: The file xtb_optimized.xyz is now available in the HPC directory,
 Terminate: False
Content successfully transferred to '/u/yunhengzou/el-agente/xtb_optimized.xyz' on the remote host.
Deleted 6 documents from 'checkpoints' collection for thread_id 'ReACT_executoreaf0d1cb-a225-405d-aa79-9a95e8caeace'.
Deleted 0 documents from 'checkpoint_writes' collection for thread_id 'ReACT_executoreaf0d1cb-a225-405d-aa79-

[AIMessage(content='Thought: I need to transfer the xyz file to the HPC for the DFT calculations., \n Action: objective: Transfer xyz file to HPC for DFT calculations., input_context: The xyz file is xtb_optimized.xyz and needs to be transferred to HPC., output: xyz file is transferred to HPC.,\n Terminate: False', additional_kwargs={}, response_metadata={}, id='ab8c258e-efcb-4e10-b134-481140093c54'),
 HumanMessage(content='Observation: The file `xtb_optimized.xyz` has been successfully transferred to the HPC directory. You can now proceed with the DFT calculations.', additional_kwargs={}, response_metadata={}, id='08d14663-34c6-4136-b6bc-9024ae47f49d'),
 AIMessage(content='Thought: I need to run the ORCA calculations on the HPC using the transferred input file and xyz file., \n Action: objective: Run ORCA calculations on HPC using the input file and xyz file., input_context: The ORCA input file and xtb_optimized.xyz file are available on HPC for DFT optimization., output: ORCA calcula

In [ ]:
manager.stream(input_node_label="DFT_Agent",query="Read the output file located at HPC.")

Agent:  DFT_Agent
Thought: I need to transfer the ORCA output file from HPC to the local directory to read it., 
 Action: objective: Transfer ORCA output file from HPC to local directory for reading., input_context: The ORCA output file is located on HPC and needs to be transferred to local for reading., output: ORCA output file is transferred to local directory.,
 Terminate: False
[File_Manager] ...
Thought: I need to transfer the ORCA output file from the HPC to the local directory to read it., 
 Action: objective: Transfer ORCA output file from HPC to local directory, input_context: The ORCA output file is named orca_output.out and is located in the HPC directory., output: The ORCA output file is transferred to the local directory.,
 Terminate: False
File '/u/yunhengzou/el-agente/orca_output.out' successfully transferred to 'orca_output.out' on the local machine.
Deleted 6 documents from 'checkpoints' collection for thread_id 'ReACT_executoreaf0d1cb-a225-405d-aa79-9a95e8caeace'.
Del

In [ ]:
python_ssh_client_mariana.close()